In [39]:
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup
import csv

# Path to the EPUB file and the output CSV file
epub_file_path = 'calendar of wisdom.epub'
output_csv_file = 'quotes.csv'


# Read EPUB content
book = epub.read_epub(epub_file_path)
content = []
for item in book.get_items():
    if item.get_type() == ebooklib.ITEM_DOCUMENT:
        content.append(item.get_body_content())

# Parse quotes and authors
quotes_data = []
for html_content in content:
    soup = BeautifulSoup(html_content, 'html.parser')
    for quote_tag in soup.find_all('p', class_='calibre15'):
        # Replace <br> tags with newlines
        for br in quote_tag.find_all('br'):
            br.replace_with('\n')

        quote = quote_tag.get_text(strip=True)

        # Find the author in the next sibling with class 'calibre16'
        author_tag = quote_tag.find_next_sibling('p', class_='calibre16')
        author = author_tag.get_text(strip=True) if author_tag else "Leo Tolstoy"

        quotes_data.append((quote, author))


for html_content in content:
    soup = BeautifulSoup(html_content, 'html.parser')
    for quote_tag in soup.find_all('blockquote'):
        quote = quote_tag.get_text(strip=True)

        # Find the author
        author_tag = quote_tag.find_next_sibling('p')
        author = author_tag.get_text(strip=True) if author_tag and author_tag.get_text(strip=True).startswith("—") else "Leo Tolstoy"

        quotes_data.append((quote, author))

# Save to CSV
with open(output_csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Quote', 'Author'])  # Writing header
    for quote, author in quotes_data:
        writer.writerow([quote, author])


In [36]:
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup

def read_epub(file_path):
    book = epub.read_epub(file_path)
    content = []
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content.append(item.get_body_content())
    return content

if __name__ == "__main__":
    epub_file_path = 'calendar of wisdom.epub'

    content = read_epub(epub_file_path)
    # Print the first 1000 characters of the first document's content to inspect the structure
    with open('content_sample.txt', 'w', encoding='utf-8') as file:
        file.write(content[6][:10000].decode('utf-8'))


In [43]:

import pandas as pd

# Read the CSV files
goodreads_quotes = pd.read_csv('goodreads_quotes.csv')
calendar_quotes = pd.read_csv('calendar_quotes.csv')

# Combine the dataframes
quotes = pd.concat([goodreads_quotes, calendar_quotes])

# Remove any rows where 'Quote' is empty or null
quotes = quotes[quotes['Quote'].notna()]

quotes['Quote'] = quotes['Quote'].apply(lambda x: x[1:-1] if (x.startswith('“') and x.endswith('”')) else x)
quotes['Quote'] = quotes['Quote'].apply(lambda x: x[1:-1] if (x.startswith('"') and x.endswith('"')) else x)
quotes = quotes[quotes['Quote'].apply(lambda x: any(c.isalpha() for c in x if c.isascii()))]
quotes.to_csv('quotes.csv', index=False)


In [44]:
x = '"I\'m selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can\'t handle me at my worst, then you sure as hell don\'t deserve me at my best."'

x.startswith('"') and x.endswith('"')

True